In [1]:
import pandas as pd
from scipy.optimize import basinhopping
from HestonModel import Heston, HestonPriceFunction

heston = Heston()
heston.load_data('Data/20230912_PLIQ_IP.csv')
heston.preprocess_data()
heston.get_from_user(risk_free_rate=.11)
heston.get_from_market()


In [2]:
strike_price = 100
spot_price = 105
yearly_historical_volatility = 0.1
riskfree_rate = 0.0
kappa = 2
epsilon = 0.1   
rho = 0.0   

HestonPriceFunction(strike_price=strike_price, spot_price=spot_price, yearly_historical_volatility = yearly_historical_volatility,
                risk_free_rate=riskfree_rate, kappa = kappa, theta=yearly_historical_volatility**2, epsilon = epsilon, rho = rho, current = "2020/8/4", maturity = "2021/8/3",
                call_option = True)

7.027599518261383